### Функция распознавания `img2table`

In [ ]:
# функция для распознавания таблиц с изображения
# pytesseract для текста + EasyOCR для недостающих (digit=0)
# или + EasyOCR для цифр (digit=1)
def img2table(img_path, output_path, digit=1):
    # Считывание изображения
    img = Image.open(img_path)
    img = np.array(img)
    if img.ndim == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Присвоение изображению порогового значения в виде двоичного изображения
    img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

    # Ширина ядра как 100-я часть общей ширины
    kernel_len = np.array(img).shape[1] // 100

    # Определение вертикального и горизонтального ядер
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

    # Ядро размером 2x2
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Обнаружение вертикальных и горизонтальных линий
    image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

    image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

    # Объединение горизонтальных и вертикальных линий в новом изображении
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)

    # Размывание и установление порогового значения
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    _, img_vh = cv2.threshold(img_vh, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Обнаружение контуров
    contours, _ = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Преобразование tuple в list
    contours = list(contours)
    contours.reverse()

    # Подсчёт вертикальных линий (столбцов)
    num_vertical_lines = 1
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if w > 20 and h > 20 and w < 500 and h < 500:
            num_vertical_lines = int(img.shape[1] / w)
            break
    
    texts = []
    # Итерация по каждому контуру (ячейке)
    for contour in contours:
        
        # Получение координат ограничивающего прямоугольника вокруг контура
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20 and w < 500 and h < 500:
            cell_image = img[y:y + h, x:x + w]
        
            # Получение текста с ячейки
            text = pytesseract.image_to_string(cell_image, lang='rus')

            if digit == 0: # вариант "EasyOCR для недостающих"
                if text == '':
                    results = reader.readtext(cell_image)
                    if len(results) > 0:
                        text = results[0][1]
            else: # вариант "EasyOCR для цифр"
                if text == '' or re.search(r'\d', text):
                    results = reader.readtext(cell_image)
                    if len(results) > 0:
                        text = results[0][1]

            # Добавление текста в список
            texts.append(text.replace('|', '').replace('_', '').replace(',', '.'))
    
    # Исключение пустых строк вначале списка
    for text in texts[num_vertical_lines:]:
        if text == '':
            texts.remove(text)
        else:
            break
        
    # Разделение списка по колучиству столбцов
    texts = [texts[i:i+num_vertical_lines] for i in range(0, len(texts), num_vertical_lines)]
    
    # Создание датафрейма
    try:
        df = pd.DataFrame(data=texts[1:], columns=texts[0])
        df = df.applymap(lambda x: x.replace('\n', ''))
        df = df.rename(columns=lambda x: re.sub(r'\s+', ' ', x.replace('\n', ' ')))
    except:
        df = pd.DataFrame()
    
    # Сохранение
    df.to_csv(output_path, index=False)

    return df